In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-227405
Azure region: eastus2
Subscription id: f9d5a085-54dc-4215-9ba6-dad5d86e60a0
Resource group: aml-quickstarts-227405


In [3]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "mimi-cluster"

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###

try:
    aml_compute = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS12_V2',
                                                           max_nodes=4)
    aml_compute = ComputeTarget.create(ws, cluster_name, compute_config)

aml_compute.wait_for_completion(show_output=True)

Found existing cluster
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [4]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
ps = RandomParameterSampling(
    {'--C': uniform(0.05, 0.5),
    '--max_iter': choice(5, 10, 50, 100)
    } ) ### YOUR CODE HERE ###

# Specify a Policy
policy = BanditPolicy(evaluation_interval=1, slack_factor=0.1) ### YOUR CODE HERE ###

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig(source_directory='.',
                      script='train.py',
                      compute_target=cluster_name,
                      environment=sklearn_env) ### YOUR CODE HERE ###

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(run_config=src,
                                     hyperparameter_sampling=ps,
                                     policy=policy,
                                     primary_metric_name='Accuracy',
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs=10) ### YOUR CODE HERE ###

In [5]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###

experiment= Experiment(ws, "hyper-mimi")
run = experiment.submit(config= hyperdrive_config)
RunDetails(run).show()
run.wait_for_completion(show_output = True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_8e18c36f-f63e-4b4d-a75b-16c5d561a5a6
Web View: https://ml.azure.com/runs/HD_8e18c36f-f63e-4b4d-a75b-16c5d561a5a6?wsid=/subscriptions/f9d5a085-54dc-4215-9ba6-dad5d86e60a0/resourcegroups/aml-quickstarts-227405/workspaces/quick-starts-ws-227405&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

[2023-03-08T14:57:28.260244][GENERATOR][INFO]Trying to sample '10' jobs from the hyperparameter space
[2023-03-08T14:57:29.0163527Z][SCHEDULER][INFO]Scheduling job, id='HD_8e18c36f-f63e-4b4d-a75b-16c5d561a5a6_1' 
[2023-03-08T14:57:28.8779518Z][SCHEDULER][INFO]Scheduling job, id='HD_8e18c36f-f63e-4b4d-a75b-16c5d561a5a6_0' 
[2023-03-08T14:57:29.1568658Z][SCHEDULER][INFO]Scheduling job, id='HD_8e18c36f-f63e-4b4d-a75b-16c5d561a5a6_2' 
[2023-03-08T14:57:29.3538950Z][SCHEDULER][INFO]Scheduling job, id='HD_8e18c36f-f63e-4b4d-a75b-16c5d561a5a6_3' 
[2023-03-08T14:57:29.4094511Z][SCHEDULER][INFO]Scheduling job, id='HD_8e18c36f-f63e-4b4d-a75b-16c5d561a5a6_4' 
[2023-03-0

{'runId': 'HD_8e18c36f-f63e-4b4d-a75b-16c5d561a5a6',
 'target': 'mimi-cluster',
 'status': 'Completed',
 'startTimeUtc': '2023-03-08T14:57:27.77409Z',
 'endTimeUtc': '2023-03-08T15:14:51.769222Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name":"Accuracy","goal":"maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '4c390083-b404-402e-b098-97b698da6582',
  'user_agent': 'python/3.8.10 (Linux-5.15.0-1031-azure-x86_64-with-glibc2.17) msrest/0.7.1 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.48.0',
  'space_size': 'infinite_space_size',
  'score': '0.9059237679048313',
  'best_child_run_id': 'HD_8e18c36f-f63e-4b4d-a75b-16c5d561a5a6_2',
  'best_metric_status': 'Succeeded',
  'best_data_container_id': 'dcid.HD_8e18c36f-f63e-4b4d-a75b-16c5d561a5a6_2'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'configuration': None,
  'attribution': None,
  'telemetryVal

KeyError: 'log_files'

In [9]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###

best_run = run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()

print(best_run)
print('Accuracy:', best_run_metrics['Accuracy'])

Run(Experiment: hyper-mimi,
Id: HD_8e18c36f-f63e-4b4d-a75b-16c5d561a5a6_2,
Type: azureml.scriptrun,
Status: Completed)
Accuracy: 0.9059237679048313


In [10]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###

ds = TabularDatasetFactory.from_delimited_files("https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv")
    

In [12]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds) ### YOUR DATA OBJECT HERE ###

train_df = x
train_df['y'] = y

In [13]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=train_df,
    label_column_name='y',
    n_cross_validations=5)

In [14]:
# Submit your automl run

### YOUR CODE HERE ###


experiment= Experiment(ws, "automl-mimi")
auto_run = experiment.submit(config= automl_config, show_output = True)

No run_configuration provided, running on local with default configuration
Running in the active local environment.


Experiment,Id,Type,Status,Details Page,Docs Page
automl-mimi,AutoML_588932c6-b3cd-48d0-8c0f-5b1c12a37a9f,automl,Preparing,Link to Azure Machine Learning studio,Link to Documentation


Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed fit featurizers and featurizing the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one cl

2023/03/08 15:53:41 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!
2023-03-08:16:16:31,689 INFO     [explanation_client.py:334] Using default datastore for uploads


In [15]:
# Get your best run and save the model from that run.

best_auto_run = auto_run.get_best_child()
print(best_auto_run)

Run(Experiment: automl-mimi,
Id: AutoML_588932c6-b3cd-48d0-8c0f-5b1c12a37a9f_22,
Type: None,
Status: Completed)
